# Retrieval Augmented Generation

## Setup API clients

In [29]:
import os
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv()

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com")

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
AZURE_SEARCH_SERVICE_KEY = os.getenv("AZURE_SEARCH_SERVICE_KEY")
search_service_cred = AzureKeyCredential(AZURE_SEARCH_SERVICE_KEY)

AZURE_SEARCH_FULL_INDEX = "gptkbindex"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=search_service_cred)


## Tìm kiếm với Semantic và Hybrid search


In [30]:
user_question = "máy tính bảng có khả năng chơi game"

In [31]:
def search(user_question):
        user_question_vector = get_embedding(user_question)
        r = search_client.search(
                user_question,
                top=5, 
                vector_queries=[
                        VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="embedding")],
                query_type="semantic",
                semantic_configuration_name="default-semantic-config")

        sources = "\n".join([f"{doc['sourcefile']}: {doc['content']}\n" for doc in r])
        return sources

result = search(user_question)
print(result)

products.txt: Canon Máy tính bảng, $1405.00, Máy tính bảng với màn hình lớn và pin lâu, phù hợp cho việc xem phim và chơi game.


products.txt: Lenovo Máy tính bảng, $1463.00, Máy tính bảng với màn hình lớn và pin lâu, phù hợp cho việc xem phim và chơi game.


products.txt: Sony Máy tính bảng, $1998.00, Máy tính bảng với màn hình lớn và pin lâu, phù hợp cho việc xem phim và chơi game.


products.txt: Nikon Máy tính bảng, $362.00, Máy tính bảng với màn hình lớn và pin lâu, phù hợp cho việc xem phim và chơi game.


products.txt: Asus Máy tính bảng, $1937.00, Máy tính bảng với màn hình lớn và pin lâu, phù hợp cho việc xem phim và chơi game.




## Gửi câu hỏi và thông tin sản phẩm tới LLM

In [32]:
SYSTEM_MESSAGE = """
Assistant helps answer customer's questions about the available products. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
user_question = "Có máy in màu không?"

sources = search(user_question)

messages = [
    {"role": "system", "content": SYSTEM_MESSAGE},
    {"role": "user", "content": user_question + "\nSources: " + sources}
]
# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_NAME,
    temperature=0.7,
    messages=messages
)

answer = response.choices[0].message.content
print("Câu hỏi: ", user_question)
print("Bot: ", answer)


Câu hỏi:  Có máy in màu không?
Bot:  Có, chúng tôi có các máy in màu sau:
- Canon Máy in, $901.00 [products.txt].
- Dell Máy in, $1393.00 [products.txt].
- Nikon Máy in, $923.00 [products.txt].
- Dell Máy in, $1720.00 [products.txt].
- Acer Máy in, $590.00 [products.txt].


### Streaming
Khi gọi với *streaming* API sẽ trả về các **ChatCompletionChunk** liên tục theo thời gian thực mà không chờ cho nội dung hoàn thành. Chúng ta cần thu thập các chunk này thành câu trả lời hoàn chỉnh.

In [33]:
user_question = "có máy in màu không?"

sources = search(user_question)

messages = [
    {"role": "system", "content": SYSTEM_MESSAGE},
    {"role": "user", "content": user_question + "\nSources: " + sources}
]
# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_NAME,
    temperature=0.7,
    messages=messages,
    stream=True
)

answer = ""
for item in response:
    print(item)
    for choice in item.choices:
        answer += choice.delta.content if choice.delta.content else ""

print(answer)

ChatCompletionChunk(id='', choices=[], created=0, model='', object='', service_tier=None, system_fingerprint=None, usage=None, prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'custom_blocklists': [], 'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'profanity': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
ChatCompletionChunk(id='chatcmpl-A80SmoJW8Gw9pFiiQgJmIrEJyWLmP', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None, content_filter_results={})], created=1726471768, model='gpt-4o-2024-05-13', object='chat.completion.chunk', service_tier=None, system_fingerprint='fp_80a1bad4c7', usage=None)
ChatCompletionChunk(id='chatcmpl-A80SmoJW8Gw9pFiiQgJmIrEJyWL

### Exercise: Cải thiện để chat với multi-turn conversation

In [34]:
# Code for handling multi-turn conversation